In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [0]:
data = pd.read_csv("indo_dataset.csv")

In [0]:
data.iloc[0]

index                                                       0
sentimen                                                    0
Tweet       lagu bosan apa yang aku save ni huhuhuhuhuhuhu...
Name: 0, dtype: object

In [0]:
max_fatures = 2500
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Tweet'].values)

In [0]:
X = tokenizer.texts_to_sequences(data['Tweet'].values)

In [0]:
X[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,  629,  983,   15,    3,
          1, 1735,   17], dtype=int32)

In [0]:
X = pad_sequences(X)

In [0]:
Y = pd.get_dummies(data['sentimen']).values

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)

In [0]:
embed_dim = 128

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(LSTM(256, ))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 25, 128)           320000    
_________________________________________________________________
lstm_13 (LSTM)               (None, 256)               394240    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 64)                16448     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 130       
Total params: 730,818
Trainable params: 730,818
Non-trainable params: 0
_______________________________________________

In [0]:
batch_size = 32
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = 10, batch_size=batch_size)

Train on 3670 samples, validate on 1809 samples
Epoch 1/10
3670/3670 [==============================] - 16s 4ms/sample - loss: 0.6692 - accuracy: 0.5891 - val_loss: 0.6025 - val_accuracy: 0.6893
Epoch 2/10
3670/3670 [==============================] - 14s 4ms/sample - loss: 0.4782 - accuracy: 0.7907 - val_loss: 0.5734 - val_accuracy: 0.7098
Epoch 3/10
3670/3670 [==============================] - 14s 4ms/sample - loss: 0.3408 - accuracy: 0.8575 - val_loss: 0.6345 - val_accuracy: 0.7137
Epoch 4/10
3670/3670 [==============================] - 14s 4ms/sample - loss: 0.2520 - accuracy: 0.8997 - val_loss: 0.7811 - val_accuracy: 0.7197
Epoch 5/10
3670/3670 [==============================] - 13s 4ms/sample - loss: 0.1834 - accuracy: 0.9259 - val_loss: 0.9790 - val_accuracy: 0.7236
Epoch 6/10
3670/3670 [==============================] - 14s 4ms/sample - loss: 0.1450 - accuracy: 0.9447 - val_loss: 1.3168 - val_accuracy: 0.7120
Epoch 7/10
3670/3670 [==============================] - 13s 4ms/sample

In [0]:
X_T = tokenizer.texts_to_sequences(np.array(["saya benci baso", "saya suka baso"]))
X_T = pad_sequences(X_T, maxlen=25)

In [0]:
for d in np.argmax(model.predict(X_T), axis=0):
  print(["Negativ", "Positif"][d])

Negativ
Positif


In [0]:
model.save_weights('lstm.h5')

In [0]:
model2 = Sequential()
model2.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model2.add(LSTM(256))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(2, activation='softmax'))
model2.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model2.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 25, 128)           320000    
_________________________________________________________________
lstm_11 (LSTM)               (None, 256)               394240    
_________________________________________________________________
dense_13 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 130       
Total params: 730,818
Trainable params: 730,818
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model2.load_weights('lstm.h5')

In [0]:
for d in np.argmax(model2.predict(X_T), axis=0):
  print(["Negativ", "Positif"][d])

Negativ
Positif
